**Yogesh Bhandare** *06*

Assignment 6 Reinforcement Learning

In [ ]:
import numpy as np

In [ ]:
# 1. define the environment (maze)
# r: regular space, g: goal (reward +10), s: start, w: wall (reward -10)
# map the environment to a 4x4 grid of rewards for the agent
environment_rewards = np.array([
    [ 0,  0,  0,  0],
    [ 0, -10,  0,  0],
    [ 0,  0,  0,  0],
    [ 0,  0,  0, 10] # goal state is 3,3 (reward 10)
])

In [ ]:
# 2. initialize q-table
# q-table has dimensions: (number of states) x (number of actions)
# states: 16 (4x4 grid cells), actions: 4 (up, down, left, right)
q_table = np.zeros((16, 4))

# map actions to indices: 0: up, 1: down, 2: left, 3: right
actions = {0: 'up', 1: 'down', 2: 'left', 3: 'right'}

In [ ]:
# 3. define hyperparameters
gamma = 0.9      # discount factor (importance of future rewards)
alpha = 0.1      # learning rate (how much new info overrides old)
epsilon = 0.1    # exploration rate (chance to take a random action)
num_episodes = 1000

In [ ]:
# 4. helper functions for navigation and state
def state_to_row_col(state):
    # converts a 1d state index (0-15) to a 2d (row, col)
    return state // 4, state % 4

def get_next_state(row, col, action):
    # determines the new (row, col) after an action
    new_row, new_col = row, col
    if action == 0: new_row = max(0, row - 1)  # up
    elif action == 1: new_row = min(3, row + 1)  # down
    elif action == 2: new_col = max(0, col - 1)  # left
    elif action == 3: new_col = min(3, col + 1)  # right

    # check for wall (the reward array handles the wall penalty)
    if environment_rewards[new_row, new_col] == -10:
        return row * 4 + col # stay in the same state if it's a wall

    return new_row * 4 + new_col # new state index

def get_reward(row, col):
    # returns reward from the environment array
    return environment_rewards[row, col]

In [ ]:
# 5. q-learning training loop
for episode in range(num_episodes):
    current_state = 0  # start at state 0 (row 0, col 0)

    # continue until the goal state (15) is reached
    while current_state != 15:
        # epsilon-greedy strategy: explore (random action) or exploit (best action)
        if np.random.uniform(0, 1) < epsilon:
            action = np.random.randint(4)  # explore: choose a random action
        else:
            # exploit: choose the action with the max q-value for the current state
            action = np.argmax(q_table[current_state, :])

        # calculate next state and reward
        row, col = state_to_row_col(current_state)
        next_state = get_next_state(row, col, action)
        reward = get_reward(*state_to_row_col(next_state))

        # q-learning formula update: q(s,a) = q(s,a) + alpha * [reward + gamma * max(q(s',a')) - q(s,a)]
        old_q = q_table[current_state, action]
        max_future_q = np.max(q_table[next_state, :])

        new_q = old_q + alpha * (reward + gamma * max_future_q - old_q)
        q_table[current_state, action] = new_q

        current_state = next_state

In [ ]:
# 6. determine and print the final path
print("q-table training complete.")
print("--- optimal path ---")
current_state = 0
path = [current_state]
while current_state != 15:
    # exploit the learned knowledge (choose the best action)
    best_action = np.argmax(q_table[current_state, :])
    row, col = state_to_row_col(current_state)
    current_state = get_next_state(row, col, best_action)
    path.append(current_state)

# map state indices back to (row, col) for visualization
path_coordinates = [state_to_row_col(s) for s in path]
print(f"path (states 0-15): {path}")
print(f"path (row, col): {path_coordinates}")

# visualize the maze path
maze_map = np.full((4, 4), 'R', dtype=str)
maze_map[0, 0] = 'S'  # start
maze_map[1, 1] = 'W'  # wall
maze_map[3, 3] = 'G'  # goal
for r, c in path_coordinates:
    if maze_map[r, c] == 'R':
        maze_map[r, c] = '*' # mark path
    elif maze_map[r, c] == 'W':
        pass # don't overwrite wall
print("\n--- maze with optimal path (*) ---")
print(maze_map)